### Estructura de archivos para la práctica 3 de Análisis de Grafos (tema: red de personajes Marvel)

Justificación: Se usa la API de Marvel para construir un grafo basado en cómics y personajes.
Cada personaje será un nodo, y los enlaces entre ellos se crean si aparecen en el mismo cómic.

In [10]:
# 01_extractor.ipynb (descarga datos desde la API de Marvel)
import requests
import pandas as pd
import time
import os

In [ ]:
# Crear carpeta INPUT si no existe
if not os.path.exists("INPUT"):
    os.makedirs("INPUT")

In [11]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve the keys
PUBLIC_KEY = os.getenv("PUBLIC_KEY")
PRIVATE_KEY = os.getenv("PRIVATE_KEY")
HASH = os.getenv("HASH")

In [12]:
BASE_URL = "https://gateway.marvel.com/v1/public"
ts = 1

In [13]:
comics = []

# Extraer hasta un número máximo de cómics (por ejemplo, 100 cómics)
max_comics = 100
limit = 20

for offset in range(0, max_comics, limit):
    url = f"{BASE_URL}/comics?orderBy=-modified&limit={limit}&offset={offset}&ts={ts}&apikey={PUBLIC_KEY}&hash={HASH}"
    response = requests.get(url)
    if response.status_code != 200:
        continue
    results = response.json()["data"]["results"]
    for comic in results:
        comic_id = comic["id"]
        title = comic["title"]
        char_url = f"{BASE_URL}/comics/{comic_id}/characters?ts={ts}&apikey={PUBLIC_KEY}&hash={HASH}"
        char_response = requests.get(char_url)
        if char_response.status_code != 200:
            continue
        characters = char_response.json()["data"]["results"]
        for char in characters:
            comics.append({
                "comic": title,
                "character": char["name"]
            })
    time.sleep(1)  # para evitar sobrecarga

In [15]:
# Guardar en INPUT
marvel_df = pd.DataFrame(comics)
marvel_df.to_csv("INPUT/marvel_characters.csv", index=False)